# **Práctica 6: Transformaciones**

<img src ="https://epigijon.uniovi.es/image/image_gallery?uuid=903ae5c8-b29b-430e-980d-1a19a885c736&groupId=3743853&t=1688576582973" width=300 px>

Este cuaderno desarrolla contenidos prácticos de la asignatura **Visión artificial** del Grado en Ciencia e Ingeniería de Datos.

***

# Implementación

La transformación geométrica en imágenes es un campo fundamental en el procesamiento de imágenes y la visión por computador, abordando la modificación de la posición, orientación y escala de objetos en una imagen. Estas transformaciones, como traslación, rotación y escalado, son cruciales para corregir distorsiones, alinear imágenes, realinear objetos y simular perspectivas. Estas operaciones permiten adaptar la geometría de las imágenes, facilitando diversas aplicaciones, desde la corrección de imperfecciones hasta la manipulación creativa de contenidos visuales.

Para realizar transformaciones en un espacio 2D a menudo es conveniente trabajar con coordenadas homogéneas. Las coordenadas homogéneas son una extensión del sistema de coordenadas euclidiano que facilita ciertas operaciones matriciales, como las utilizadas en transformaciones geométricas.

> Implementa una función para convertir puntos 2D a coordenadas homogéneas. Como entrada se reciben un array de tamaño $2 \times N$, donde $N$ es el número de puntos. La salida es un array de tamaño $3 \times N$ añadiendo una fila de unos.

> Implementa una función para convertir coordenadas homogéneas a puntos 2D. Como entrada se reciben un array de tamaño $3 \times N$, donde $N$ es el número de puntos. La salida es un array de tamaño $2 \times N$ tras dividir las dos primeras filas entre la tercera.

> Añade tests usando `np.testing.assert_allclose` para verificar el correcto funcionamiento de las funciones.

> Define un conjunto de puntos y verifica su transformación. Utiliza como ejemplo el siguiente [demostrador](https://iis.uibk.ac.at/public/piater/courses/demos/homography/homography.xhtml)

In [1]:
import numpy as np

def to_homogeneous(points: np.ndarray) -> np.ndarray:
    """
    Convierte puntos 2D a coordenadas homogéneas.
    
    Args:
        points (np.ndarray): Array de tamaño (2, N) con N puntos 2D.
    
    Returns:
        np.ndarray: Array de tamaño (3, N) con coordenadas homogéneas.
    """
    if points.shape[0] != 2:
        raise ValueError("La entrada debe tener un tamaño de (2, N)")
    
    ones = np.ones((1, points.shape[1]))
    return np.vstack([points, ones])

def from_homogeneous(homogeneous_points: np.ndarray) -> np.ndarray:
    """
    Convierte coordenadas homogéneas a puntos 2D.
    
    Args:
        homogeneous_points (np.ndarray): Array de tamaño (3, N) con N puntos homogéneos.
    
    Returns:
        np.ndarray: Array de tamaño (2, N) con coordenadas 2D.
    """
    if homogeneous_points.shape[0] != 3:
        raise ValueError("La entrada debe tener un tamaño de (3, N)")
    
    return homogeneous_points[:2] / homogeneous_points[2]

# Pruebas con np.testing.assert_allclose
def test_homogeneous_transformations():
    points_2d = np.array([[1, 2, 3], [4, 5, 6]])  # (2,3)
    expected_homogeneous = np.array([[1, 2, 3], [4, 5, 6], [1, 1, 1]])  # (3,3)
    
    # Prueba de conversión a homogéneas
    np.testing.assert_allclose(to_homogeneous(points_2d), expected_homogeneous)
    
    # Prueba de conversión desde homogéneas
    np.testing.assert_allclose(from_homogeneous(expected_homogeneous), points_2d)
    
    print("Todas las pruebas pasaron correctamente.")

# Ejecutar pruebas
test_homogeneous_transformations()


Todas las pruebas pasaron correctamente.


> Implementa una función que permita realizar la transformación geométrica de una imagen `def transform_image(img, matrix)` siguiendo los siguientes pasos:

> Crea un array con las coordenadas de los límites de la imagen (esquina superior izquierda, derecha, etc.).
>
> Convierte el array a coordenadas homogéneas, realiza la transformación usando la multiplicación de matrices, y determina los límites mínimos y máximos de las esquinas transformadas.
>
> Crea un array de puntos entre los punto mínimo y máximo en cada eje usando `np.linspace`. El número de elementos debe ser proporcional al tamaño de la imagen en cada dimensión.
>
> Crea un grid usando `np.meshgrid`.
>
> Transforma las coordenadas del grid usando la transformación inversa.
>
> Para cada canal de la imagen, reinterpola la imagen en las coordenadas transformadas usando `scipy.ndimage.map_coordinates`. Esta función recibe las coordenadas en este orden: primero las filas y luego las columnas. Los datos que retorna está función será necesario convertirlos en matriz usando `reshape`.
>
> La función debe retornar la imagen transformada y las dimensiones (`[xmin, xmax, ymax, ymin]`)
>
> Visualiza la imagen usando `imshow(warped, extent=extent)`



In [2]:
import scipy

def transform_image(img: np.ndarray, matrix: np.ndarray):
    h, w = img.shape[:2]
    
    # Coordenadas de las esquinas de la imagen
    corners = np.array([[0, w, w, 0], [0, 0, h, h]])
    
    # Convertir a coordenadas homogéneas y transformar
    transformed_corners = from_homogeneous(matrix @ to_homogeneous(corners))
    
    # Determinar los límites de la imagen transformada
    xmin, xmax = transformed_corners[0].min(), transformed_corners[0].max()
    ymin, ymax = transformed_corners[1].min(), transformed_corners[1].max()
    
    # Crear un grid de coordenadas en la imagen transformada
    new_x = np.linspace(xmin, xmax, w)
    new_y = np.linspace(ymin, ymax, h)
    grid_x, grid_y = np.meshgrid(new_x, new_y)
    
    # Convertir a coordenadas homogéneas e invertir la transformación
    grid_homogeneous = to_homogeneous(np.vstack([grid_x.ravel(), grid_y.ravel()]))
    inverse_matrix = np.linalg.inv(matrix)
    original_coords = from_homogeneous(inverse_matrix @ grid_homogeneous)
    
    # Interpolar cada canal de la imagen original
    warped = np.zeros((h, w, img.shape[2]), dtype=img.dtype)
    for i in range(img.shape[2]):
        warped[..., i] = scipy.ndimage.map_coordinates(
            img[..., i], [original_coords[1], original_coords[0]], order=1, mode='constant'
        ).reshape(h, w)
    
    extent = [xmin, xmax, ymax, ymin]
    return warped, extent

In [3]:
import matplotlib
matplotlib.use('TkAgg')  
import matplotlib.pyplot as plt

def visualize_transformed_image(img: np.ndarray, matrix: np.ndarray):
    warped, extent = transform_image(img, matrix)
    plt.imshow(warped, extent=extent)
    plt.show()


> Implementa ejemplos de traslaciones, rotaciones, proyecciones y combinaciones de estas.



In [4]:
# Definición de transformaciones básicas
def translation_matrix(tx, ty):
    return np.array([[1, 0, tx], [0, 1, ty], [0, 0, 1]])

def rotation_matrix(theta):
    return np.array([[np.cos(theta), -np.sin(theta), 0], 
                     [np.sin(theta), np.cos(theta), 0], 
                     [0, 0, 1]])

def scaling_matrix(sx, sy):
    return np.array([[sx, 0, 0], [0, sy, 0], [0, 0, 1]])

def projection_matrix(px, py):
    return np.array([[1, 0, 0], [0, 1, 0], [px, py, 1]])


transformations = {
        "Translation": translation_matrix(50, 30),
        "Rotation": rotation_matrix(np.pi / 6),
        "Scaling": scaling_matrix(1.5, 0.8),
        "Projection": projection_matrix(0.001, 0.001),
        "Combination": translation_matrix(50, 30) @ rotation_matrix(np.pi / 6) @ scaling_matrix(1.2, 1.2)
    }



In [7]:
import skimage


img= skimage.io.imread("s1.jpg")

for name, matrix in transformations.items():
        plt.figure()
        plt.title(name)
        visualize_transformed_image(img, matrix)
plt.show(block=True)

visualize_transformed_image(img,[[1.35,0.59,100],[0.32,2.31,0],[6.1e-04,1.3e-03,1]])

# Estimación

La estimación de una transformación se realiza comúnmente utilizando métodos de mínimos cuadrados para minimizar la diferencia entre las coordenadas transformadas y las coordenadas reales de los puntos correspondientes.

Como ejemplo, se va a estimar una transformación para obtener una imagen frontal de un cuadro.

In [14]:
!curl http://www.atc.uniovi.es/grado/3va/prac/pareja.png -o pareja.png

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2862k  100 2862k    0     0  11.0M      0 --:--:-- --:--:-- --:--:-- 11.2M


In [7]:
import skimage.io
from skimage import img_as_float
img = skimage.io.imread("pareja.png")
img = img_as_float(img)
plt.imshow(img)
plt.show()

En este caso, se proporcionan las coordenadas en la imagen en píxeles y las de destino en milímetros.

In [8]:
src = np.array(
    [
        [ 240,  218, 1219, 1159],
        [ 410, 1498,  212, 1765]
    ]
)
W = 699
H = 813
dst = np.array(
    [
        [  0,   0, 699, 699],
        [  0, 813,   0, 813]
    ]
)
plt.imshow(img)
plt.plot(src[0,:], src[1,:], '.r')
plt.show()

Para estimar la transformación, dados los puntos en la imagen $(x_i, y_i)$ y sus correspondencias en coordenadas del mundo $(x_i', y_i')$. Se debe resolver el sistema de ecuaciones:

$$
\begin{bmatrix}
x_1 & y_1 & 1 & 0 & 0 & 0 & -x_1' x_1 & -x_1' y_1 & -x'_1 \\
0 & 0 & 0 & x_1 & y_1 & 1 & -y'_1 x_1 & -y'_1 y_1 & -y'_1 \\
x_2 & y_2 & 1 & 0 & 0 & 0 & -x_2' x_2 & -x_2' y_2 & -x'_2 \\
0 & 0 & 0 & x_2 & y_2 & 1 & -y'_2 x_2 & -y'_2 y_2 & -y'_2 \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots\\
x_n & y_n & 1 & 0 & 0 & 0 & -x_n' x_n & -x_n' y_n & -x'_n \\
0 & 0 & 0 & x_n & y_n & 1 & -y'_n x_n & -y'_n y_n & -y'_n \\
\end{bmatrix}
\begin{bmatrix}
    h_{11} \\
    h_{12} \\
    h_{13} \\
    h_{21} \\
    h_{22} \\
    h_{23} \\
    h_{31} \\
    h_{32} \\
    h_{33}
\end{bmatrix} = \mathbf{0}
$$

> Para estimar la transformación se deben seguir los siguientes pasos:
>
> Construye la matriz de coeficientes tal y como se muestra en la ecuación anterior usando todos los puntos disponibles.
>
> Utiliza la función `scipy.linalg.svd` para obtener la descomposición de valores singulares de la matriz de coeficientes. La solución es la última fila de $V$: `V[-1, :].reshape(3, 3)`.
>
> Transforma la imagen usando el resultado anterior y verifica el resultado.
>
> Estima la transformación usando `skimage.transform.estimate_transform` y verifica que proporciona los mismos resultados.

In [9]:
# Construcción de la matriz de coeficientes
num_points = src.shape[1]
A = []

for i in range(num_points):
    x, y = src[:, i]
    x_p, y_p = dst[:, i]
    A.append([x, y, 1, 0, 0, 0, -x_p * x, -x_p * y, -x_p])
    A.append([0, 0, 0, x, y, 1, -y_p * x, -y_p * y, -y_p])

A = np.array(A)

# Resolver el sistema usando SVD
_, _, V = scipy.linalg.svd(A)
V = V[-1, :].reshape(3, 3)

# Mostrar la matriz de transformación obtenida
V


array([[-2.43287917e-03, -4.91942480e-05,  6.04060643e-01],
       [-3.51500540e-04, -1.73797489e-03,  7.96929835e-01],
       [-1.04563389e-06,  2.36933065e-08, -2.12388346e-03]])

In [10]:
import scipy.ndimage

def apply_homography(img, V):
    
    h, w = img.shape[:2]
    # Definir las esquinas de la imagen original
    corners = np.array([[0, w, w, 0],
                        [0, 0, h, h],
                        [1, 1, 1, 1]])

    # Transformar las esquinas usando V
    transformed_corners = V @ corners
    transformed_corners /= transformed_corners[2]  

    # Obtener los nuevos límites de la imagen transformada
    xmin, xmax = transformed_corners[0].min(), transformed_corners[0].max()
    ymin, ymax = transformed_corners[1].min(), transformed_corners[1].max()

    # Crear una malla de coordenadas en la imagen transformada
    new_x = np.linspace(xmin, xmax, w)
    new_y = np.linspace(ymin, ymax, h)
    grid_x, grid_y = np.meshgrid(new_x, new_y)

    # Convertir a coordenadas homogéneas e invertir la transformación
    inv_H = np.linalg.inv(V)
    grid_homogeneous = np.vstack([grid_x.ravel(), grid_y.ravel(), np.ones(grid_x.size)])
    original_coords = inv_H @ grid_homogeneous
    original_coords /= original_coords[2]  # Convertir a coordenadas cartesianas

    # Interpolar la imagen original en las nuevas coordenadas
    warped = np.zeros((h, w, img.shape[2]), dtype=img.dtype) if len(img.shape) == 3 else np.zeros((h, w), dtype=img.dtype)

    for i in range(img.shape[2] if len(img.shape) == 3 else 1):
        warped[..., i] = scipy.ndimage.map_coordinates(
            img[..., i] if len(img.shape) == 3 else img,
            [original_coords[1], original_coords[0]],
            order=1, mode='constant'
        ).reshape(h, w)

    return warped, (xmin, xmax, ymin, ymax)

# Aplicar la homografía
warped_img, extent = apply_homography(img,V)

# Visualizar la imagen transformada
plt.figure(figsize=(8, 6))
plt.imshow(warped_img, extent=extent, cmap='gray' if len(img.shape) == 2 else None)
plt.title("Imagen Transformada")
plt.show()


In [11]:
from skimage.transform import estimate_transform

# Estimar la homografía usando skimage
tform = estimate_transform('projective', src.T, dst.T)

# Obtener la matriz de transformación estimada por skimage
H_skimage = tform.params

print("\nMatriz estimada con skimage:\n", H_skimage)

# Aplicar la transformación con la matriz de skimage
warped_img_skimage, extent_skimage = apply_homography(img, H_skimage)

# Visualizar la imagen transformada con skimage
plt.figure(figsize=(8, 6))
plt.imshow(warped_img_skimage, extent=extent_skimage, cmap='gray' if len(img.shape) == 2 else None)
plt.title("Imagen Transformada con skimage")
plt.show()



Matriz estimada con skimage:
 [[ 1.14548619e+00  2.31624046e-02 -2.84413271e+02]
 [ 1.65498977e-01  8.18300496e-01 -3.75222958e+02]
 [ 4.92321689e-04 -1.11556528e-05  1.00000000e+00]]
